In [1]:
import pandas as pd
import seaborn as snb
import matplotlib.pyplot as plt
import numpy as np
#import holidays
#from datetime import date
#feriados = holidays.Brazil()
#import dash
#import dash_html_components as html
#import dash_core_components as dcc
#import dash_table_experiments as dt
#from dash.dependencies import Input, Output
#from jupyter_dash import JupyterDash
#import plotly.graph_objects as go
#from datetime import date
#import datetime
#import plotly.express as px
import gspread

In [174]:
gc = gspread.oauth()
sd=gc.open_by_key('1uScOR2FQcq_pxAbM0o32MGbO2ABYc_s5l5hUbIqs784')
df = pd.DataFrame(sd.worksheet('Movimentações').get_all_records())
df['Preço unitário'] = df['Preço unitário'].str.replace('-','0',regex=True)
df['Preço unitário'] = df['Preço unitário'].str.replace('R\$ ','',regex=True).str.replace('.','',regex=True).str.replace(',','.').astype(float)

df['Valor da Operação'] = df['Valor da Operação'].str.replace('-','0',regex=True)
df['Valor da Operação'] = df['Valor da Operação'].str.replace('R\$ ','',regex=True).str.replace('.','',regex=True).str.replace(',','.').astype(float)

df = df.iloc[::-1]
df

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação,Ticker
435,Credito,04/12/2019,Transferência - Liquidação,PETR4 - PETROLEO BRASILEIRO S.A. PETROBRAS,INTER DTVM LTDA,14,29.44,412.16,PETR4
434,Credito,04/12/2019,Transferência - Liquidação,IVVB11 - ISHARES S&P 500 FDO INV COTAS FDO INDICE,INTER DTVM LTDA,3,143.50,430.50,IVVB11
433,Credito,04/12/2019,Transferência - Liquidação,ITUB4 - ITAU UNIBANCO HOLDING S.A.,INTER DTVM LTDA,16,34.94,559.04,ITUB4
432,Credito,04/12/2019,Transferência - Liquidação,EGIE3 - ENGIE BRASIL ENERGIA S.A.,INTER DTVM LTDA,11,47.35,520.85,EGIE3
431,Credito,04/12/2019,Transferência - Liquidação,CRFB3 - ATACADÃO S.A.,INTER DTVM LTDA,17,18.76,318.92,CRFB3
...,...,...,...,...,...,...,...,...,...
4,Credito,25/10/2021,Rendimento,XPML11 - XP MALLS FUNDO DE INVESTIMENTO IMOBIL...,INTER DTVM LTDA,50,0.57,28.50,XPML11
3,Credito,25/10/2021,Atualização,VBBR3 - PETROBRAS DISTRIBUIDORA S/A,INTER DTVM LTDA,163,0.00,0.00,VBBR3
2,Credito,25/10/2021,Rendimento,SNFF11 - SUNO FUNDO DE FUNDOS DE INVESTIMENTO ...,INTER DTVM LTDA,43,0.70,30.10,SNFF11
1,Credito,01/11/2021,Dividendo,ITUB4 - ITAU UNIBANCO HOLDING S/A,INTER DTVM LTDA,102,0.02,1.53,ITUB4


In [197]:
medio = 0
quantidade = 0
investido = 0
lucro = 0
dividendos = 0
investimentos = pd.DataFrame(columns=['Ticker','Quantidade','Preço médio','Total investido','Dividendos'])
for ticker in df['Ticker'].unique():
    medio = 0
    quantidade = 0
    investido = 0
    dividendos = 0
    for i in  df[(df['Movimentação'].isin(['Transferência - Liquidação','Desdobro','Bonificação em Ativos','Atualização','Incorporação']))&(df['Ticker']==ticker)].itertuples():
        if i[1]=='Credito':
            investido = investido + i[8]
            quantidade = quantidade + i[6]
            medio = investido/quantidade
        else:
            quantidade = quantidade - i[6]
            lucro = lucro + (i[8]-i[6]*medio)
            if quantidade ==0:
                investido = medio=0
            else:
                investido = quantidade*medio
                medio = investido/quantidade
    print (ticker,investido,quantidade, medio)
    for i in df[(df['Movimentação'].isin(['Dividendo','Rendimento','Juros Sobre Capital Próprio']))&(df['Ticker']==ticker)].itertuples():
        dividendos = dividendos + i[8]

    if quantidade != 0:
        investimentos = investimentos.append({'Ticker':ticker,'Quantidade':quantidade,'Preço médio':medio,'Total investido':investido,'Dividendos':dividendos},ignore_index=True)
print(lucro)

PETR4 0 0 0
IVVB11 2483.0 13 191.0
ITUB4 2795.01 102 27.402058823529416
EGIE3 4135.14 100 41.351400000000005
CRFB3 0 0 0
SHUL4 3949.15 549 7.193351548269582
B3SA3 9857.56 600 16.429266666666667
VVAR3 0 0 0
TAEE11 3175.37 100 31.7537
MGLU3 1847.5 80 23.09375
LAME3 0 0 0
FLRY3 7304.639999999999 300 24.348799999999997
WIZS3 0 0 0
BRDT3 2799.04 163 17.1720245398773
HGLG11 4086.4 24 170.26666666666668
HCTR11 9631.82 81 118.91135802469135
BCFF11 5297.28 60 88.288
MRFG3 2115.0 101 20.940594059405942
BIDI4 2120.0 300 7.066666666666666
ITSA4 5097.3099999999995 500 10.194619999999999
BCFF12 0 0 0
HCTR12 0 0 0
HCTR14 0 0 0
BCFF13 0 0 0
XPCM11 0 0 0
BIDI2 0 0 0
XPML11 5135.7 50 102.714
SOMA3 2230.0 200 11.15
IFIE13 800.0 8 100.0
IFIE11 1792.6 18 99.58888888888889
RBRD11 5422.5 85 63.794117647058826
IRDM11 3018.14 26 116.0823076923077
MULT3 4280.0 200 21.4
BRML3 1824.0 400 4.56
TIET11 4371.3 262 16.684351145038168
IRBR3 1196.1 178 6.719662921348314
HCTR13 0 0 0
XPLG11 5790.0 50 115.8
IRDM12 0 0 0
I

In [198]:
investimentos

,Ticker,Quantidade,Preço médio,Total investido,Dividendos
0,IVVB11,13,191.000000,2483.00,0
1,ITUB4,102,27.402059,2795.01,77.63
2,EGIE3,100,41.351400,4135.14,255.98
3,SHUL4,549,7.193352,3949.15,97.2
4,B3SA3,600,16.429267,9857.56,411.49
5,TAEE11,100,31.753700,3175.37,394.5
6,MGLU3,80,23.093750,1847.50,1.12
7,FLRY3,300,24.348800,7304.64,243.2
8,BRDT3,163,17.172025,2799.04,243.46
9,HGLG11,24,170.266667,4086.40,195.68


In [202]:
sd.worksheet('Totais').batch_clear(["A1:E"])
sd.worksheet('Totais').update([investimentos.columns.values.tolist()]+investimentos.values.tolist())

{'spreadsheetId': '1uScOR2FQcq_pxAbM0o32MGbO2ABYc_s5l5hUbIqs784',
 'updatedRange': 'Totais!A1:E33',
 'updatedRows': 33,
 'updatedColumns': 5,
 'updatedCells': 165}

In [187]:
    medio = 0
    quantidade = 0
    investido = 0
    dividendos = 0
    for i in  df[(df['Movimentação'].isin(['Transferência - Liquidação','Desdobro','Bonificação em Ativos','Atualização']))&(df['Ticker']=='AESB3')].itertuples():
        if i[1]=='Credito':
            investido = investido + i[8]
            quantidade = quantidade + i[6]
            medio = investido/quantidade
        else:
            quantidade = quantidade - i[6]
            lucro = lucro + (i[8]-i[6]*medio)
            if quantidade ==0:
                investido = medio=0
            else:
                investido = quantidade*medio
                medio = investido/quantidade
print(medio,quantidade, investido)

0 0 0
